In [1]:
from pydantic import BaseModel, field_serializer

In [2]:
from datetime import datetime

In [3]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="always")
    def serialize_name(self, value):
        print(f"type={type(value)}")
        return value

In [4]:
m = Model(dt="2020-01-01T12:00:00")
m

Model(dt=datetime.datetime(2020, 1, 1, 12, 0))

In [5]:
m.model_dump()

type=<class 'datetime.datetime'>


{'dt': datetime.datetime(2020, 1, 1, 12, 0)}

In [6]:
m.model_dump_json()

type=<class 'datetime.datetime'>


'{"dt":"2020-01-01T12:00:00"}'

In [7]:
m = Model(dt=None)

In [8]:
m.model_dump()

type=<class 'NoneType'>


{'dt': None}

In [9]:
m.model_dump_json()

type=<class 'NoneType'>


'{"dt":null}'

In [10]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="unless-none")
    def serialize_name(self, value):
        print(f"type={type(value)}")
        return value

In [11]:
m = Model(dt=None)


In [12]:
m.model_dump()

{'dt': None}

In [13]:
m.model_dump_json()

'{"dt":null}'

In [14]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="json-unless-none")
    def serialize_name(self, value):
        print(f"type={type(value)}")
        return value.strftime("%Y/%-m/%-d %I:%M %p")

In [15]:
m = Model(dt="2020-01-01T12:00:00")
m

Model(dt=datetime.datetime(2020, 1, 1, 12, 0))

In [16]:
m.model_dump_json()

type=<class 'datetime.datetime'>


'{"dt":"2020/1/1 12:00 PM"}'

In [17]:
from pydantic import FieldSerializationInfo

In [18]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="unless-none")
    def dt_serializer(self, value, info: FieldSerializationInfo):
        print(f"info={info}")
        return value

In [19]:
m = Model(dt="2020-01-01T12:00:00")
m

Model(dt=datetime.datetime(2020, 1, 1, 12, 0))

In [20]:
m.model_dump()

info=SerializationInfo(include=None, exclude=None, context=None, mode='python', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, exclude_computed_fields=False, round_trip=False, serialize_as_any=False)


{'dt': datetime.datetime(2020, 1, 1, 12, 0)}

In [21]:
m.model_dump_json()

info=SerializationInfo(include=None, exclude=None, context=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, exclude_computed_fields=False, round_trip=False, serialize_as_any=False)


'{"dt":"2020-01-01T12:00:00"}'

In [25]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="unless-none")
    def dt_serializer(self, value, info: FieldSerializationInfo):
        print(f"mode_is_json={info.mode_is_json()}")
        return value

In [26]:
m = Model(dt=datetime(2020, 1, 1))
m.model_dump()

mode_is_json=False


{'dt': datetime.datetime(2020, 1, 1, 0, 0)}

In [27]:
m.model_dump_json()

mode_is_json=True


'{"dt":"2020-01-01T00:00:00"}'

In [29]:
import pytz

def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

In [30]:
dt = make_utc(datetime.now())
dt

datetime.datetime(2025, 12, 16, 3, 43, 23, 94951, tzinfo=<UTC>)

In [31]:
dt.isoformat()

'2025-12-16T03:43:23.094951+00:00'

In [32]:
dt.strftime("%Y-%m-%dT%H:%M:%SZ")

'2025-12-16T03:43:23Z'

In [33]:
def dt_utc_json_serializer(dt: datetime) -> str:
    dt = make_utc(dt)
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")

In [34]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="unless-none")
    def dt_serializer(self, dt, info: FieldSerializationInfo):
        if info.mode_is_json():
            return dt_utc_json_serializer(dt)
        return make_utc(dt)

In [35]:
m = Model(dt=datetime(2020, 1, 1))
m

Model(dt=datetime.datetime(2020, 1, 1, 0, 0))

In [36]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>)}

In [37]:
m.model_dump_json()

'{"dt":"2020-01-01T00:00:00Z"}'

In [38]:
eastern = pytz.timezone('US/Eastern')
dt = eastern.localize(datetime(2020, 1, 1))
dt

datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [39]:
m = Model(dt=dt)
m

Model(dt=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>))

In [40]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 5, 0, tzinfo=<UTC>)}

In [41]:
m.model_dump_json()

'{"dt":"2020-01-01T05:00:00Z"}'